In [32]:
!pip install 'litgpt[all]' datasets seqeval

In [2]:
from litgpt import LLM

llm = LLM.load("microsoft/phi-2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Setting HF_HUB_ENABLE_HF_TRANSFER=1


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

Converting .safetensor files to PyTorch binaries (.bin)
checkpoints/microsoft/phi-2/model-00002-of-00002.safetensors --> checkpoints/microsoft/phi-2/model-00002-of-00002.bin
checkpoints/microsoft/phi-2/model-00001-of-00002.safetensors --> checkpoints/microsoft/phi-2/model-00001-of-00002.bin
Converting checkpoint files to LitGPT format.
{'checkpoint_dir': PosixPath('checkpoints/microsoft/phi-2'),
 'debug_mode': False,
 'dtype': None,
 'model_name': None}


Loading weights: model-00002-of-00002.bin: 100%|██████████| 00:54<00:00,  1.85it/s


Saving converted checkpoint to checkpoints/microsoft/phi-2


In [3]:
text = llm.generate("Correct the spelling: Every summer, the familly enjoys a trip to the mountains.")
print(text)

 The boys are on their way to school.



In [3]:
import pandas as pd
import numpy as np
from datasets import load_dataset

# Load the dataset from Hugging Face
dataset = load_dataset("gtfintechlab/finer-ord")

# Convert each split to a DataFrame
train_df = pd.DataFrame(dataset['train'])
val_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])

README.md:   0%|          | 0.00/5.34k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

val.csv:   0%|          | 0.00/135k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/336k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/80531 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10233 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25957 [00:00<?, ? examples/s]

In [4]:
def group_by_sentence(df):
    """
    Converts token-level DataFrame to sentence-level DataFrame where each row contains
    just the sentence tokens and corresponding labels.

    Args:
        df (pd.DataFrame): DataFrame with columns ['gold_label', 'gold_token', 'doc_idx', 'sent_idx']

    Returns:
        pd.DataFrame: DataFrame with columns ['tokens', 'labels']
    """
    # Convert None values to the string "None" in gold_token column
    df['gold_token'] = df['gold_token'].fillna("None")

    # Group by both document and sentence indices
    grouped = df.groupby(['doc_idx', 'sent_idx']).agg({
        'gold_token': list,
        'gold_label': list
    }).reset_index()

    # Keep only the relevant columns and rename them
    return grouped[['gold_token', 'gold_label']].rename(columns={
        'gold_token': 'words',
        'gold_label': 'labels'
    })

train_df = group_by_sentence(train_df)
val_df = group_by_sentence(val_df)
test_df = group_by_sentence(test_df)

In [5]:
len(test_df)

1075

In [6]:
train_df.head()

,words,labels
0,"[Kenyan, Firms, Eye, Deals, During, Obama, Sum...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[By, Neville, Otuki, Kenya, 's, business, lead...","[0, 1, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[Industrialists, ,, entrepreneurs, and, banker...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[More, on, This, Kenya, :, Mombasa, Road, ,, U...","[0, 0, 0, 3, 0, 3, 4, 0, 3, 4, 0, 3, 4, 4, 4, ..."
4,"["", Local, manufacturers, will, be, looking, f...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
import pandas as pd
import json

def convert_to_litgpt_format(df, id2label):
    dataset = []

    instruction = """You are an expert at named entity recoginition.
You will be presented with a sentence.
Identify the entities which classify as "Person", "Location", or "Organisation" and tag them accordingly, in the order they appear in.
Ignore the non entity words. Input sentence: """

    for idx, row in df.iterrows():
        words = row['words']
        labels = [id2label[label] for label in row['labels']]

        # Combine words into a sentence
        input_text = ' '.join(words)

        # Create structured output format
        entities = []
        current_entity = None

        for word, label in zip(words, labels):
            if label != 'O':
                if label.startswith('B-'):
                    if current_entity:
                        entities.append(current_entity)
                    current_entity = {'type': label[2:], 'text': word}
                elif label.startswith('I-') and current_entity:
                    current_entity['text'] += f" {word}"
            else:
                if current_entity:
                    entities.append(current_entity)
                    current_entity = None

        if current_entity:
            entities.append(current_entity)

        # Format output as a structured string
        output_text = ""
        for entity in entities:
            output_text += f"{entity['type']}: {entity['text']}\n"

        if not output_text:
            output_text = "No named entities found."

        dataset.append({
            "instruction": instruction,
            "input": input_text,
            "output": output_text.strip()
        })

    return dataset

In [7]:
# Convert and save
id2label = {
        0: 'O',
        1: 'B-PER', 2: 'I-PER',
        3: 'B-LOC', 4: 'I-LOC',
        5: 'B-ORG', 6: 'I-ORG'
    }
litgpt_dataset = convert_to_litgpt_format(train_df, id2label)

# Preview first example
print("Sample entry:")
print(json.dumps(litgpt_dataset[0], indent=2))

Sample entry:
{
  "instruction": "You are an expert at named entity recoginition. \nYou will be presented with a sentence. \nIdentify the entities which classify as \"Person\", \"Location\", or \"Organisation\" and tag them accordingly, in the order they appear in. \nIgnore the non entity words. Input sentence: ",
  "input": "Kenyan Firms Eye Deals During Obama Summit Tagged : The Global Entrepreneurship Summit , launched by President Obama in 2009 , brings together entrepreneurs and investors from across Africa and around the world annually to showcase innovative projects , exchange new ideas , and help spur economic opportunity .",
  "output": "PER: Obama\nPER: Obama\nLOC: Africa"
}


In [8]:
with open('ner_dataset.json', 'w') as f:
    json.dump(litgpt_dataset, f, indent=2)

In [9]:
!litgpt finetune_lora microsoft/phi-1_5 \
    --data JSON \
    --data.json_path ner_dataset.json \
    --data.val_split_fraction 0 \
    --quantize "bnb.nf4" \
    --precision "bf16-true" \
    --out_dir out/ner-model \
    --train.micro_batch_size 1 \
    --train.max_seq_length 512 \
    --train.epochs 3

Streaming output truncated to the last 5000 lines.
Epoch 2 | iter 4863 step 303 | loss train: 0.695, val: nan | iter time: 600.29 ms
Epoch 2 | iter 4864 step 304 | loss train: 0.700, val: nan | iter time: 572.56 ms (step)
Epoch 2 | iter 4865 step 304 | loss train: 0.703, val: nan | iter time: 447.93 ms
Epoch 2 | iter 4866 step 304 | loss train: 0.673, val: nan | iter time: 561.25 ms
Epoch 2 | iter 4867 step 304 | loss train: 0.713, val: nan | iter time: 714.85 ms
Epoch 2 | iter 4868 step 304 | loss train: 0.743, val: nan | iter time: 565.68 ms
Epoch 2 | iter 4869 step 304 | loss train: 0.722, val: nan | iter time: 552.87 ms
Epoch 2 | iter 4870 step 304 | loss train: 0.688, val: nan | iter time: 563.83 ms
Epoch 2 | iter 4871 step 304 | loss train: 0.677, val: nan | iter time: 573.41 ms
Epoch 2 | iter 4872 step 304 | loss train: 0.657, val: nan | iter time: 555.47 ms
Epoch 2 | iter 4873 step 304 | loss train: 0.655, val: nan | iter time: 610.52 ms
Epoch 2 | iter 4874 step 304 | loss trai

In [10]:
llm = LLM.load("out/ner-model/final")

In [16]:
val_df.head()

,words,labels
0,"[Tip, :, Use, comma, (, ,, ), to, separate, mu...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[Learn, more, ...]","[0, 0, 0]"
2,"[Tue, ,, Oct, 20, ,, 2015, ,, 03:57, BST, -, UK]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5]"
3,"[Markets, open, in, 4, hrs, 3, mins, Asia, dro...","[6, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[The, Canadian, dollar, ,, already, under, pre...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [46]:
test_prompt = """You are an expert at named entity recoginition.
You will be presented with a sentence.
Identify the entities which classify as "PER", "LOC", or "ORG" and tag them accordingly, in the order they appear in.
Ignore the non entity words. Input sentence: """ + ' '.join(train_df.iloc[0]['words'])

print(test_prompt)

You are an expert at named entity recoginition. 
You will be presented with a sentence. 
Identify the entities which classify as "Person", "Location", or "Organisation" and tag them accordingly, in the order they appear in. 
Ignore the non entity words. Input sentence: Kenyan Firms Eye Deals During Obama Summit Tagged : The Global Entrepreneurship Summit , launched by President Obama in 2009 , brings together entrepreneurs and investors from across Africa and around the world annually to showcase innovative projects , exchange new ideas , and help spur economic opportunity .


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy the model folder to your Google Drive
!cp -r out/ner-model/final "/content/drive/My Drive/finer-finetuned-phi-1_5-v1"

In [49]:
from seqeval.metrics import f1_score
import re

def evaluate_ner_performance(llm, test_df, id2label):
    base_prompt = """You are an expert at named entity recoginition.
You will be presented with a sentence.
Identify the entities which classify as "Person", "Location", or "Organisation" and tag them accordingly, in the order they appear in.
Ignore the non entity words. Input sentence: """
    # Lists to store all predictions and ground truth
    all_true_labels = []
    all_pred_labels = []

    # Process each sentence
    for idx, row in test_df.iterrows():
        # Get the sentence and true labels
        sentence = ' '.join(row['words'])
        true_labels = [id2label[label] for label in row['labels']]

        # Format prompt with current sentence
        prompt = base_prompt + sentence

        # Get model prediction
        output = llm.generate(prompt)

        # Parse model output to extract predictions
        pred_labels = parse_llm_output(output, row['words'])

        # Store for evaluation
        all_true_labels.append(true_labels)
        all_pred_labels.append(pred_labels)

        # Optional: Print progress
        if (idx + 1) % 100 == 0:
            print(f"Processed {idx + 1} sentences")

    # Calculate and return f1 score
    f1 = f1_score(all_true_labels, all_pred_labels)
    return f1

def parse_llm_output(output, words):
    """Parse LLM output into list of labels matching seqeval format."""
    # Initialize labels list
    pred_labels = []

    # Expected format: "tag:tokens"
    try:
        # Split output into lines and process each token:label pair
        pairs = re.findall(r'(\w+):(\w+(?:\*[BI])?)', output)

        # Create mapping of predictions
        pred_mapping = {}
        for token, label in pairs:
            if label == 'Other':
                pred_mapping[token] = 'O'
            else:
                # Convert format from "Person*B" to "B-PER"
                entity_type, suffix = label.split('*') if '*' in label else (label, 'B')
                if entity_type == 'Person':
                    entity_type = 'PER'
                elif entity_type == 'Location':
                    entity_type = 'LOC'
                elif entity_type == 'Organisation':
                    entity_type = 'ORG'

                pred_mapping[token] = f'{suffix}-{entity_type}'

        # Map predictions to original words, default to 'O' if not found
        for word in words:
            pred_labels.append(pred_mapping.get(word, 'O'))

    except Exception as e:
        print(f"Error parsing output: {e}")
        print(f"Raw output: {output}")
        # If parsing fails, return all 'O' labels
        pred_labels = ['O'] * len(words)

    return pred_labels

# Example usage:
# f1 = evaluate_ner_performance(llm, test_df, id2label)
# print(f"F1 Score: {f1:.4f}")